In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
os.listdir()

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#lets load the data
data = pd.read_csv('../input/lego-minifigures-classification/index.csv')
data.head(10)

In [ ]:
#lets check abt the data df
data.shape

In [ ]:
data['class_id'].value_counts()

In [ ]:
data.isnull().sum()

In [ ]:
import cv2
import matplotlib.pyplot as plt

we dont have null values..good for us

In [ ]:
for path in data['path']:
    img = cv2.imread(str('../input/lego-minifigures-classification'+'/'+path))
    #plt.imshow(img)
    print(np.array(img))
    print(np.array(img)/255)
    break

In [ ]:
#lets load all the images
X = []
y = []

for path in data['path']:
    img = cv2.imread(str('../input/lego-minifigures-classification'+'/'+path))
    #print(img)
    resized_img = cv2.resize(img,(224,224))
    #print(resized_img)
    scaled_img = np.array(resized_img)/255
    X.append(scaled_img)
    
y = np.array(data['class_id'])-1

In [ ]:
X = np.array(X)
y = np.array(y)

In [ ]:
X[0]

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,y,random_state=0)

In [ ]:
print(f'train size : {x_train.shape}')
print(f'test size : {x_test.shape}')

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,Sequential


In [ ]:
data_augmentation = Sequential([
    layers.experimental.preprocessing.RandomFlip('horizontal',input_shape = (512,512,3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
])

In [ ]:
print(len(data['class_id'].value_counts()))
print(len(data['class_id'].unique()))

In [ ]:
num_classes = len(data['class_id'].value_counts())

In [ ]:
#model building
model = Sequential([
    data_augmentation,
    layers.Conv2D(64,3,padding='same',activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32,3,padding='same',activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(16,3,padding='same',activation='relu'),
    layers.Flatten(),
    layers.Dense(128,activation='relu'),
    layers.Dense(num_classes),
])

In [ ]:
model.compile(
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
optimizer = 'adam',
metrics = ['accuracy'],
)

In [ ]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=50,verbose=2)

In [ ]:
#lets try transfer learning methods
model_1 = tf.keras.applications.DenseNet121()
base_input = model_1.layers[0].input
base_output = model_1.layers[-2].output
output = layers.Dense(num_classes,activation='softmax')(base_output)
new_model = keras.Model(inputs = base_input,outputs = output)
print(new_model.summary())

In [ ]:
new_model.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(),optimizer ='adam',metrics=['accuracy'])

In [ ]:
new_model.fit(x_train,y_train,epochs = 50,validation_data=(x_test,y_test))

In [ ]:
image = cv2.imread('../input/lego-minifigures-classification/marvel/0001/001.jpg')
# read the image 
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = cv2.resize(image, dsize=(224,224))/255
plt.imshow(image)
image = np.reshape(image,(1,224,224,3))
ans = new_model.predict(image).argmax()
ans = ans+1
metadata = pd.read_csv('../input/lego-minifigures-classification/metadata.csv')
name = metadata['minifigure_name'][metadata['class_id']==ans].iloc[0]
print(f'minifigure name : {name}')

In [ ]:
metadata = pd.read_csv('../input/lego-minifigures-classification/metadata.csv')
metadata['minifigure_name'][metadata['class_id']==1].iloc[0]

In [ ]:
metadata